<a href="https://colab.research.google.com/github/JZanataNeto/chatbot-banks/blob/main/chatbot_banks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [9]:
################################################
# --- Agente 1: Planejador de marketing --- #
################################################
def agente_planejador(topico):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de marketing, especialista em produtos bancários. Com base nas respostas do cliente, você deverá:
        1. usar a ferramenta de busca google_search para oferecer os produtos bancários adequados ao cliente;
        2. tomar como exempo as seguintes dicas:
        - Se  resp1 for sim, ofertar um financiamento imobiliário;
        - Se  resp1 for não, ofertar seguro residencial;
        - Se  resp2 for sim, ofertar seguro de veículos;
        - Se  resp2 for não, ofertar consórcio de veículos;
        - Se  resp3 for sim, ofertar seguro de vida;
        - Se  resp4 for sim, ofertar seguro de vida;
        - Se  resp5 for não, ofertar plano odontológico;
        - Se  resp6 for sim, ofertar portabilidade de crédito;
        - Se  resp6 for não, ofertar empréstimos;
        - Se  resp7 for sim, ofertar inventimentos mais rentáveis;
        - Se  resp7 for não, ofertar investimentos;
        - Se  resp8 for sim, ofertar cartões de crédito com mais vantagens;
        - Se  resp8 for não, ofertar cartões de crédito;
        - Se  resp9 for sim, ofertar portabilidade de plano de previdência;
        - Se  resp9 for não, ofertar plano de previdência;
        - Se  resp10 for sim, ofertar planos de capitalização;
        - Se  resp11 for sim, ofertar câmbio;
        - Se  resp12 for sim, ofertar câmbio;
        - Se  resp13 for sim, ofertar travels cards.

        """,
        description="Agente que planeja marketing",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}"
    # Executa o agente
    plano_de_marketing = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_marketing

In [10]:
######################################
# --- Agente 2: Redator dos Cards --- #
######################################
def agente_redator(topico, plano_de_marketing):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar cards para produtos bancários.
            Você escreve cards para auxiliar o atendente bancário a conhecer os pontos mais importantes de cada produto selecionado.
            Os cards devem fornecer informações sobre planos de pagamento, benefícios e condições especiais.
            O card deve acrescentar informações de como adquirir o produto bancário pelo app do banco.
            O card deve ser informativo, com linguagem simples e de fácil leitura.
            """,
        description="Agente redator de cards",
        tools=[google_search]
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de marketing: {plano_de_marketing}"
    # Executa o agente
    cards = call_agent(redator, entrada_do_agente_redator)
    return cards

In [11]:
##########################################
# --- Agente 3: Marketing Manager --- #
##########################################
def agente_marketing(topico, cards):
    manager = Agent(
        name="agente_marketing",
        model="gemini-2.0-flash",
        instruction="""
            Você é um profissional de marketing, especializado em vendas de produtos bancários.
            Para alcançar um público maior, utilize uma linguagem mais informal.
            Para cada produto selecionado, crie argumentos de vendas que mostrem as vantagens e benefícios do produto.
            Os argumentos serão frases motivacionais a serem utilizadas pelos atendentes bancários de modo a convencer o cliente a contratar o produto.
            Os argumentos poderão ser indiretos ou diretos. Por exemplo: realize seus sonhos juntando dinheiro todo mês(indireto); com capitalização, você concorre a prêmios todo mês(direto).
            """,
        description="Agente gerenciador de marketing.",
        tools=[google_search]
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nCards: {cards}"
    # Executa o agente
    argumentos = call_agent(manager, entrada_do_agente_revisor)
    return argumentos

In [12]:
print("🚀 Iniciando o assitente de negócios com 3 Agentes 🚀")

# --- Obter o Tópico do atendente bancário ---
resp1 = input("O cliente possui Casa própria?")
resp2 = input("O cliente possui carro próprio?")
resp3 = input("O cliente possui esposa?")
resp4 = input("O cliente possui filhos?")
resp5 = input("o cliente possui plano odontológico?")
resp6 = input("O cliente possui empréstimos bancários em outro banco?")
resp7 = input("O cliente possui investimentos em outros bancos?")
resp8 = input("O cliente possui cartão de crédito em outros bancos?")
resp9 = input("O cliente possui um plano de previdência privada em outro banco?")
resp10 = input("O cliente gosta de fazer apostas em loterias?")
resp11 = input("O cliente possui parentes que residem no estrangeiro?")
resp12 = input("O cliente faz compras internacionais?")
resp13 = input("O cliente usa cartão de crédito em viagens internacionais?")

# Constrói a string 'topico' a partir das respostas do usuário
topico = f"""
Respostas do cliente:
Casa própria: {resp1}
Carro próprio: {resp2}
Esposa: {resp3}
Filhos: {resp4}
Plano odontológico: {resp5}
Empréstimos em outro banco: {resp6}
Investimentos em outros bancos: {resp7}
Cartão de crédito em outros bancos: {resp8}
Plano de previdência privada em outro banco: {resp9}
Gosta de apostas em loterias: {resp10}
Parentes no estrangeiro: {resp11}
Compras internacionais: {resp12}
Usa cartão em viagens internacionais: {resp13}
"""

plano_de_marketing = agente_planejador(topico)
print("\n---Resultado do Agente 1---\n")
display(to_markdown(plano_de_marketing))
print("-----------------------------------------------------")

cards = agente_redator(topico, plano_de_marketing)
print("\n---Resultado do Agente 2---\n")
display(to_markdown(cards))
print("-----------------------------------------------------")

argumentos = agente_marketing(topico, cards)
print("\n---Resultado do Agente 3---\n")
display(to_markdown(argumentos))
print("-----------------------------------------------------")

🚀 Iniciando o assitente de negócios com 3 Agentes 🚀
O cliente possui Casa própria?sim
O cliente possui carro próprio?não
O cliente possui esposa?sim
O cliente possui filhos?sim
o cliente possui plano odontológico?não
O cliente possui empréstimos bancários em outro banco?sim
O cliente possui investimentos em outros bancos?sim
O cliente possui cartão de crédito em outros bancos?sim
O cliente possui um plano de previdência privada em outro banco?sim
O cliente gosta de fazer apostas em loterias?sim
O cliente possui parentes que residem no estrangeiro?não
O cliente faz compras internacionais?sim
O cliente usa cartão de crédito em viagens internacionais?sim

---Resultado do Agente 1---



> Com base nas suas respostas, preparei algumas ofertas de produtos bancários que podem ser do seu interesse:
> 
> *   **Financiamento Imobiliário:** Como você mencionou que tem casa própria, um financiamento imobiliário pode ser interessante para investir em reformas, ampliações ou até mesmo adquirir um novo imóvel para investimento.
> 
> *   **Consórcio de Veículos:** Já que você não possui carro próprio, um consórcio de veículos pode ser uma ótima opção para planejar a compra do seu próximo carro de forma parcelada e sem juros.
> 
> *   **Seguro de Vida:** Por ter esposa e filhos, um seguro de vida pode garantir a segurança financeira da sua família em caso de imprevistos.
> 
> *   **Plano Odontológico:** Para complementar seus cuidados com a saúde, um plano odontológico pode oferecer acesso a diversos tratamentos e consultas a preços acessíveis.
> 
> *   **Portabilidade de Crédito:** Se você possui empréstimos em outro banco, a portabilidade de crédito pode trazer taxas de juros mais vantajosas e condições de pagamento melhores.
> 
> *   **Investimentos Mais Rentáveis:** Já que você possui investimentos em outros bancos, podemos analisar juntos opções de investimentos mais rentáveis e adequadas ao seu perfil de risco.
> 
> *   **Cartões de Crédito com Mais Vantagens:** Se você já utiliza cartões de crédito de outros bancos, podemos apresentar opções com mais vantagens, como programas de recompensas, cashback e benefícios exclusivos.
> 
> *   **Portabilidade de Plano de Previdência:** Por ter um plano de previdência privada em outro banco, a portabilidade pode trazer benefícios como taxas menores e melhor rentabilidade.
> 
> *   **Planos de Capitalização:** Para quem gosta de apostas em loterias, os planos de capitalização podem ser uma alternativa interessante para guardar dinheiro e concorrer a prêmios.
> 
> *   **Câmbio:** Como você realiza compras internacionais, podemos oferecer serviços de câmbio com taxas competitivas e diversas opções de moedas estrangeiras.
> 
> *   **Travel Cards:** Por usar cartão em viagens internacionais, os travel cards podem te ajudar a economizar e ter mais segurança durante suas viagens, com taxas de câmbio fixas e proteção contra perdas e roubos.
> 
> Para entender melhor suas necessidades e apresentar as melhores opções, podemos agendar uma conversa?
> 
> 
> Com base nas informações que você forneceu e nas ofertas de produtos bancários disponíveis, elaborei algumas recomendações adicionais que podem otimizar ainda mais a sua situação financeira:
> 
> *   **Financiamento Imobiliário:** Avalie as opções de financiamento imobiliário da Caixa Econômica Federal, que frequentemente oferece as taxas mais baixas e programas habitacionais acessíveis. Outros bancos como o Bradesco e Itaú Unibanco também podem ser considerados.
> 
> *   **Consórcio de Veículos:** Ao optar por um consórcio, lembre-se de que a contemplação pode ocorrer por sorteio ou lance. Se tiver pressa para adquirir o veículo, planeje oferecer lances para aumentar suas chances de ser contemplado mais rapidamente.
> 
> *   **Seguro de Vida:** Compare diferentes opções de seguro de vida para encontrar a cobertura mais adequada às necessidades da sua família. Verifique se o seguro cobre eventos como falecimento, invalidez por acidente ou doença, e se oferece assistência funeral.
> 
> *   **Plano Odontológico:** Pesquise planos odontológicos que ofereçam uma ampla rede de dentistas credenciados e cobertura para os procedimentos que você e sua família mais utilizam, como consultas, limpezas, radiografias e tratamentos de canal.
> 
> *   **Portabilidade de Crédito:** Antes de solicitar a portabilidade, compare as taxas de juros e condições de pagamento oferecidas por diferentes bancos para garantir que a transferência seja realmente vantajosa.
> 
> *   **Investimentos Mais Rentáveis:** Consulte um especialista em investimentos para identificar as opções mais adequadas ao seu perfil de risco e objetivos financeiros. Diversifique seus investimentos para reduzir os riscos e aumentar as chances de obter bons resultados.
> 
> *   **Cartões de Crédito com Mais Vantagens:** Avalie os programas de recompensas, cashback e benefícios exclusivos oferecidos por diferentes cartões de crédito para escolher aquele que melhor se adapta aos seus hábitos de consumo.
> 
> *   **Portabilidade de Plano de Previdência:** Analise as taxas de administração, rentabilidade e opções de investimento oferecidas por diferentes planos de previdência para verificar se a portabilidade pode trazer benefícios significativos.
> 
> *   **Planos de Capitalização:** Entenda como funcionam os planos de capitalização e avalie se eles são adequados aos seus objetivos financeiros. Lembre-se de que a rentabilidade desses planos costuma ser baixa e que parte do valor pago é destinada ao custeio dos sorteios.
> 
> *   **Câmbio:** Compare as taxas de câmbio oferecidas por diferentes instituições financeiras e casas de câmbio para garantir que você está obtendo o melhor valor ao comprar ou vender moedas estrangeiras.
> 
> *   **Travel Cards:** Verifique as taxas de câmbio, tarifas e benefícios oferecidos por diferentes travel cards para escolher aquele que melhor se adapta às suas necessidades e orçamento de viagem.
> 
> Se precisar de mais informações sobre algum desses produtos ou serviços, ou se tiver alguma dúvida, estou à disposição para ajudar!


-----------------------------------------------------

---Resultado do Agente 2---



> Perfeito! Aqui estão alguns cards informativos sobre os produtos bancários mencionados, elaborados para auxiliar o atendente bancário:
> 
> **Card 1: Financiamento Imobiliário**
> 
> *   **O que é:** Crédito para você investir em reformas, ampliações ou adquirir um novo imóvel (inclusive para investimento!).
> *   **Planos de Pagamento:** Diversas opções de parcelamento, com as menores taxas do mercado e programas habitacionais acessíveis.
> *   **Como Adquirir:** Simule e contrate pelo app do banco ou agende um horário com seu gerente.
> *   **Dica:** A Caixa Econômica Federal frequentemente oferece as taxas mais competitivas!
> 
> **Card 2: Consórcio de Veículos**
> 
> *   **O que é:** Planeje a compra do seu carro novo de forma parcelada e sem juros.
> *   **Benefícios:** Flexibilidade para usar seu crédito, poder de compra à vista e possibilidade de ser contemplado por sorteio ou lance.
> *   **Como Adquirir:** Simule e contrate pelo app do banco ou agende um horário com seu gerente.
> *   **Dica:** Se tiver pressa, planeje lances para aumentar suas chances de contemplação!
> 
> **Card 3: Seguro de Vida**
> 
> *   **O que é:** Proteção financeira para sua família em caso de imprevistos.
> *   **Coberturas:** Falecimento, invalidez por acidente ou doença, assistência funeral e mais.
> *   **Como Adquirir:** Simule e contrate pelo app do banco ou agende um horário com seu gerente.
> *   **Dica:** Compare diferentes opções para encontrar a cobertura ideal para as necessidades da sua família!
> 
> **Card 4: Plano Odontológico**
> 
> *   **O que é:** Acesso a diversos tratamentos e consultas a preços acessíveis.
> *   **Coberturas:** Consultas, limpezas, radiografias, tratamentos de canal e mais.
> *   **Como Adquirir:** Simule e contrate pelo app do banco ou agende um horário com seu gerente.
> *   **Dica:** Verifique a rede de dentistas credenciados e as coberturas que você e sua família mais utilizam!
> 
> **Card 5: Portabilidade de Crédito**
> 
> *   **O que é:** Traga seus empréstimos de outros bancos para cá e consiga taxas de juros mais vantajosas e melhores condições de pagamento.
> *   **Como Adquirir:** Faça uma simulação pelo app ou fale com seu gerente para analisar as melhores opções.
> *   **Dica:** Compare as taxas e condições de pagamento antes de solicitar a portabilidade!
> 
> **Card 6: Investimentos Mais Rentáveis**
> 
> *   **O que é:** Encontre opções de investimentos mais rentáveis e adequadas ao seu perfil de risco.
> *   **Benefícios:** Diversificação, potencialização de ganhos e segurança para o seu futuro.
> *   **Como Adquirir:** Converse com seu gerente para uma análise personalizada ou explore as opções no app do banco.
> *   **Dica:** Consulte um especialista para identificar as opções mais adequadas aos seus objetivos financeiros!
> 
> **Card 7: Cartões de Crédito com Mais Vantagens**
> 
> *   **O que é:** Cartões com programas de recompensas, cashback, benefícios exclusivos e muito mais!
> *   **Como Adquirir:** Compare as opções no app do banco ou fale com seu gerente para encontrar o cartão ideal para você.
> *   **Dica:** Avalie os programas de recompensas e benefícios que melhor se adaptam aos seus hábitos de consumo!
> 
> **Card 8: Portabilidade de Plano de Previdência**
> 
> *   **O que é:** Traga seu plano de previdência privada de outro banco e aproveite taxas menores e melhor rentabilidade.
> *   **Como Adquirir:** Converse com seu gerente para analisar as opções e simular os benefícios da portabilidade.
> *   **Dica:** Analise as taxas de administração, rentabilidade e opções de investimento antes de decidir!
> 
> **Card 9: Planos de Capitalização**
> 
> *   **O que é:** Guarde dinheiro e concorra a prêmios!
> *   **Como Adquirir:** Simule e contrate pelo app do banco ou agende um horário com seu gerente.
> *   **Dica:** Entenda como funcionam os planos e avalie se eles são adequados aos seus objetivos financeiros. Lembre-se que a rentabilidade costuma ser baixa.
> 
> **Card 10: Câmbio**
> 
> *   **O que é:** Serviços de câmbio com taxas competitivas e diversas opções de moedas estrangeiras.
> *   **Como Adquirir:** Consulte as taxas e realize suas operações de câmbio pelo app do banco ou em uma agência.
> *   **Dica:** Compare as taxas de diferentes instituições para garantir o melhor valor!
> 
> **Card 11: Travel Cards**
> 
> *   **O que é:** Economize e tenha mais segurança em suas viagens internacionais, com taxas de câmbio fixas e proteção contra perdas e roubos.
> *   **Como Adquirir:** Solicite seu travel card pelo app do banco ou em uma agência.
> *   **Dica:** Verifique as taxas, tarifas e benefícios oferecidos por diferentes travel cards para escolher o melhor para você!
> 


-----------------------------------------------------

---Resultado do Agente 3---



> Com base nas respostas do cliente e nos cards de produtos bancários, aqui estão algumas sugestões de frases motivacionais para o atendente bancário, visando oferecer os produtos mais adequados:
> 
> **1. Consórcio de Veículos:**
> 
> *   "Que tal planejar a compra do seu próximo carro sem juros? Com o consórcio, você pode ser sorteado ou dar um lance para acelerar a conquista!"
> *   "Já pensou em trocar de carro sem se endividar com juros? O consórcio é uma maneira inteligente de realizar esse sonho!"
> 
> **2. Seguro de Vida:**
> 
> *   "Sua família é seu bem mais precioso, certo? Garanta a segurança e o futuro deles com um seguro de vida completo e personalizado."
> *   "Imprevistos acontecem, mas você pode estar preparado. Com o seguro de vida, sua família estará protegida financeiramente em qualquer situação."
> 
> **3. Plano Odontológico:**
> 
> *   "Sorrir é essencial para a felicidade! Com o plano odontológico, você e sua família terão acesso aos melhores tratamentos e cuidados com um preço acessível."
> *   "Cuidar da saúde bucal é investir na sua qualidade de vida. Com o plano odontológico, você terá acesso a consultas, limpezas e tratamentos sem se preocupar com os custos."
> 
> **4. Portabilidade de Crédito:**
> 
> *   "Está pagando juros altos em outros bancos? Traga seus empréstimos para cá e economize dinheiro com taxas menores e condições de pagamento mais vantajosas!"
> *   "Que tal organizar suas finanças e pagar menos juros? Com a portabilidade de crédito, você pode ter uma folga no orçamento e ainda realizar seus sonhos!"
> 
> **5. Investimentos Mais Rentáveis:**
> 
> *   "Já pensou em fazer seu dinheiro render mais? Invista conosco e conquiste seus objetivos financeiros com segurança e rentabilidade."
> *   "O futuro te espera! Comece a investir agora e garanta uma aposentadoria tranquila e uma vida financeira estável."
> 
> **6. Cartões de Crédito com Mais Vantagens:**
> 
> *   "Aproveite ao máximo suas compras com um cartão cheio de benefícios! Cashback, programas de recompensas, descontos exclusivos... Qual deles combina mais com você?"
> *   "Viaje, compre, divirta-se e ainda ganhe recompensas! Com nossos cartões de crédito, você aproveita a vida ao máximo e ainda tem vantagens exclusivas."
> 
> **7. Portabilidade de Plano de Previdência:**
> 
> *   "Está satisfeito com a rentabilidade do seu plano de previdência? Traga-o para cá e veja seu dinheiro render mais com taxas menores e melhores opções de investimento!"
> *   "Planeje o futuro com tranquilidade e segurança! Com a portabilidade do plano de previdência, você garante uma aposentadoria ainda melhor."
> 
> **8. Planos de Capitalização:**
> 
> *   "Já pensou em guardar dinheiro e ainda concorrer a prêmios incríveis? Com o plano de capitalização, você realiza seus sonhos e ainda pode ser sorteado!"
> *   "Comece a poupar de forma divertida e ainda tenha a chance de ganhar prêmios! Com o plano de capitalização, você junta dinheiro para o futuro e ainda concorre a sorteios."
> 
> **9. Câmbio:**
> 
> *   "Vai viajar para o exterior? Conte com nossos serviços de câmbio para ter as melhores taxas e a segurança que você precisa."
> 
> **10. Travel Cards:**
> 
> *   "Viaje com tranquilidade e segurança! Com o travel card, você tem taxas de câmbio fixas, proteção contra perdas e roubos e muito mais."
> *   "Economize e tenha mais controle dos seus gastos em viagens internacionais com o travel card. A solução ideal para quem ama explorar o mundo!"
> 


-----------------------------------------------------
